In [11]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

from multiprocessing import Pool
from itertools import product
import knntsp
import warnings
import importlib
warnings.filterwarnings('ignore')

NUM_CORES = 4
PREPROCESSED_PATH = './../preprocessed/'

In [12]:
# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

In [13]:
# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'median_timeseries.csv')

In [14]:
# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)

In [15]:
W_TO_TEST = [3,4,5] #, 27
K_TO_TEST = [1,2,3,4,5]
RANDOM_STATE = 1
TRAIN_SIZE = 0.86
FOR_TEST_PREDICTION = 20

In [16]:
importlib.reload(knntsp)

FRAC = 0.1

results = []
X = timeSeries.iloc[:-FOR_TEST_PREDICTION]#timeSeries.T.sample(frac=FRAC, replace=False, random_state=RANDOM_STATE).T.copy().iloc[:-FOR_TEST_PREDICTION]

to_run = []
for w in W_TO_TEST:
    for k in K_TO_TEST:

        for column in X.columns:
            to_run.append([X, column, w, k, TRAIN_SIZE])
            
with Pool(NUM_CORES) as pool:
    results = pool.starmap(knntsp.predict_with_knn, to_run)

In [19]:
SAVE_SIMPLE_W_K_SELECTION = False

simple_w_k_result = pd.DataFrame()
for result in results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    simple_w_k_result = simple_w_k_result.append(pd.Series([w, k, mean_error[0]]), ignore_index=True)

simple_w_k_result.columns = ['w','k', 'mean_error']
# groupped_mean = simple_w_k_result.groupby(['w','k'])['mean_error'].mean()
# groupped_mean
simple_w_k_result
# if SAVE_SIMPLE_W_K_SELECTION:
#     pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error']).to_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')

# pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error'])

,w,k,mean_error
0,3.0,1.0,0.630144
1,3.0,1.0,0.541282
2,3.0,1.0,0.639017
3,3.0,1.0,0.321319
4,3.0,1.0,0.348579
5,3.0,1.0,0.608084
6,3.0,1.0,0.470213
7,3.0,1.0,0.394829
8,3.0,1.0,0.347385
9,3.0,1.0,0.473567


In [ ]:
indexes = pd.Series([value*len(simple_w_k_result) for value in range(0, 16)])
simple_final_result = pd.DataFrame()
i = 0
for i in range(0, len(simple_w_k_result)):
    simple_final_result = simple_final_result.append(simple_w_k_result.loc[simple_w_k_result.loc[indexes]['mean_error'].idxmin()])
    indexes += 1
simple_final_result.reset_index(inplace=True)
simple_final_result.shape

In [ ]:
SAVE_SIMPLE_W_K_SELECTION = True

if SAVE_SIMPLE_W_K_SELECTION:
    simple_final_result.to_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')


In [8]:
importlib.reload(knntsp)
                 
cluster_labels = pd.read_csv(PREPROCESSED_PATH + 'median_based_cluster_labels.csv', header=-1, index_col=[0])
cluster_labels.columns = ['label']
cluster_labels.shape

CLUSTER_FRAC = 0.2

to_run = []
clustered_results = []
for name, group in cluster_labels.groupby(['label']):    
    group_sample = group.sample(frac=CLUSTER_FRAC, replace=False, random_state=RANDOM_STATE).copy()
    cluster_X = timeSeries.T.iloc[group_sample.index].T.iloc[:FOR_TEST_PREDICTION]    
    for w in W_TO_TEST:
        for k in K_TO_TEST:
            for column in cluster_X.columns:#[(16,21)]
                to_run.append([cluster_X, column, w, k, TRAIN_SIZE, False, name])

with Pool(NUM_CORES) as pool:
    clustered_results = pool.starmap(knntsp.predict_with_knn, to_run)

In [9]:
SAVE_CLUSTER_W_K_SELECTION = True

cluster_w_k_result = pd.DataFrame()
for result in clustered_results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    cluster_w_k_result = cluster_w_k_result.append(pd.Series([cluster_label, w, k, mean_error[0]]), ignore_index=True)

cluster_w_k_result.columns = ['cluster_label', 'w','k', 'mean_error']
cluster_groupped_mean = cluster_w_k_result.groupby(['cluster_label', 'w','k'], as_index=False)['mean_error'].mean()
cluster_groupped_mean = cluster_groupped_mean.iloc[cluster_groupped_mean.groupby(['cluster_label'])['mean_error'].idxmin().values]
cluster_groupped_mean.sort_values('cluster_label', inplace=True)
cluster_groupped_mean.reset_index(drop=True, inplace=True)

if SAVE_CLUSTER_W_K_SELECTION:
    cluster_groupped_mean.to_csv(PREPROCESSED_PATH + 'cluster_knntsp_w_k.csv')

cluster_groupped_mean.shape

(90, 4)

In [10]:
cluster_groupped_mean

,cluster_label,w,k,mean_error
0,1.0,4.0,5.0,0.066486
1,2.0,4.0,3.0,0.529674
2,3.0,5.0,2.0,0.779532
3,4.0,3.0,1.0,1.948583
4,5.0,3.0,2.0,1.238267
5,6.0,5.0,5.0,0.236962
6,7.0,3.0,2.0,0.653356
7,8.0,5.0,5.0,0.744913
8,9.0,5.0,5.0,0.760080
9,10.0,3.0,5.0,0.817274


In [ ]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

from multiprocessing import Pool
from itertools import product
import knntsp
import warnings
import importlib
warnings.filterwarnings('ignore')

import pickle

NUM_CORES = 4
PREPROCESSED_PATH = './../preprocessed/'
RESULTS_PATH = './../results/'

# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

NUM_CORES = 4
RANDOM_STATE = 1
TRAIN_SIZE = 0.86

# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'median_timeseries.csv')

# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)

cluster_labels = pd.read_csv(PREPROCESSED_PATH + 'median_based_cluster_labels.csv', header=-1, index_col=[0])
cluster_labels.columns = ['label']
cluster_labels.head()

simple_w_k = pd.read_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv',index_col=[0])
cluster_w_k = pd.read_csv(PREPROCESSED_PATH + 'cluster_knntsp_w_k.csv')

simple_w_k.loc[0]['w']

importlib.reload(knntsp)

results_cluster_knn_real = []
results_cluster_knn_predicted = []

results_simple_knn_real = []
results_simple_knn_predicted = []

to_run_cluster_real = []
to_run_cluster_predicted = []

to_run_simple_real = []
to_run_simple_predicted = []

X = timeSeries.copy()
_cluster_labels = cluster_labels.copy()

i = 0
for column in X.columns:
    label = cluster_labels.iloc[X.T.index.get_loc(column)][0]
    
    clust_w_k = cluster_w_k[cluster_w_k.cluster_label==label]
    w_k = simple_w_k.loc[i]
#     if len(clust_w_k) == 0:
#         clust_w_k = simple_w_k
    
    to_run_cluster_real.append([X, column, _cluster_labels, int(clust_w_k.iloc[0]['w']), int(clust_w_k.iloc[0]['k']), TRAIN_SIZE, False, True])
    to_run_cluster_predicted.append([X, column, _cluster_labels, int(clust_w_k.iloc[0]['w']), int(clust_w_k.iloc[0]['k']), TRAIN_SIZE, False, False])
    
    to_run_simple_predicted.append([X, column, int(w_k['w']), int(w_k['k']), TRAIN_SIZE, False, label, False])
    to_run_simple_real.append([X, column, int(w_k['w']), int(w_k['k']), TRAIN_SIZE, False, label, True])

with Pool(NUM_CORES) as pool:
    results_cluster_knn_real = pool.starmap(knntsp.predict_with_cluster_knn, to_run_cluster_real)
    results_simple_knn_real = pool.starmap(knntsp.predict_with_knn, to_run_simple_real)
    results_cluster_knn_predicted = pool.starmap(knntsp.predict_with_cluster_knn, to_run_cluster_predicted)
    results_simple_knn_predicted = pool.starmap(knntsp.predict_with_knn, to_run_simple_predicted)

In [18]:
SAVE_PICKLE_RESULTS = True

if SAVE_PICKLE_RESULTS:
    output = open(RESULTS_PATH + 'median_excluded_w_k_cluster_real.pk', 'wb')
    pickle.dump(results_cluster_knn_real, output)
    output.close()
    
    output = open(RESULTS_PATH + 'median_excluded_w_k_simple_real.pk', 'wb')
    pickle.dump(results_simple_knn_real, output)
    output.close()
    
    output = open(RESULTS_PATH + 'median_excluded_w_k_cluster_predicted.pk', 'wb')
    pickle.dump(results_cluster_knn_predicted, output)
    output.close()
    
    output = open(RESULTS_PATH + 'median_excluded_w_k_simple_predicted.pk', 'wb')
    pickle.dump(results_simple_knn_predicted, output)
    output.close()   

NameError: name 'RESULTS_PATH' is not defined